In [12]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
import pickle
from sklearn.model_selection import train_test_split,GridSearchCV
import tensorflow
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping,TensorBoard
import datetime


In [5]:
df = pd.read_csv('data.csv')

with open('encoder.pkl','rb') as file:
    encoder = pickle.load(file)

df['Quality'] = encoder.fit_transform(df['Quality'])


X = df.drop(columns= ['Quality'])
y = df['Quality']

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('scaler_1.pkl', 'wb') as file:
    pickle.dump(scaler,file)

with open('encoder_1.pkl', 'wb') as file:
    pickle.dump(encoder,file)

In [31]:
def create_model(neuron=64, layers=1):
    model = Sequential()
    model.add(Dense(neuron, activation='relu', input_shape=(X_train.shape[1],)))

    for _ in range(layers-1):
        model.add(Dense(neuron, activation='relu'))
    
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

In [32]:
from scikeras.wrappers import KerasClassifier
model = KerasClassifier(build_fn=create_model, neuron=16, layers=1, verbose=1)


In [33]:
param_grid = {
    'neuron': [16, 32, 64, 128],
    'layers': [1, 2, 3, 4, 5, 6],
    'epochs': [50, 100, 150, 200],
    'optimizer':[]
}

In [34]:
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))

Fitting 3 folds for each of 96 candidates, totalling 288 fits


c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/100


c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.5786 - loss: 0.6903  
Epoch 2/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 243us/step - accuracy: 0.5584 - loss: 0.6827
Epoch 3/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 276us/step - accuracy: 0.5717 - loss: 0.6740
Epoch 4/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.5487 - loss: 0.6820  
Epoch 5/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 832us/step - accuracy: 0.5807 - loss: 0.6676
Epoch 6/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.6092 - loss: 0.6436  
Epoch 7/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 0s/step - accuracy: 0.6130 - loss: 0.6427  
Epoch 8/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 218us/step - accuracy: 0.6053 - loss: 0.6425
Epoch 9/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.5710 - loss: 0.6586 
Epoch 10/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6397 - loss: 0.6205 
Epoch 11/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.6240 - loss: 0.6347 
Epoch 12/100
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accur

# Optimizing Accuracy


In [35]:
df = pd.read_csv('data.csv')

with open('encoder.pkl','rb') as file:
    encoder = pickle.load(file)

df['Quality'] = encoder.fit_transform(df['Quality'])


X = df.drop(columns= ['Quality'])
y = df['Quality']

X_train,X_test,y_train,y_test = train_test_split(X,y, test_size = 0.2, random_state=42)

with open('scaler.pkl', 'rb') as file:
    scaler = pickle.load(file)

X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

with open('scaler_1.pkl', 'wb') as file:
    pickle.dump(scaler,file)

with open('encoder_1.pkl', 'wb') as file:
    pickle.dump(encoder,file)

In [41]:
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.layers import Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam

def create_improved_model(neuron=64, layers=1, dropout_rate=0.2):
    model = Sequential()
    model.add(Dense(neuron, activation='relu', input_shape=(X_train.shape[1],)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))

    for _ in range(layers-1):
        model.add(Dense(neuron, activation='relu'))
        model.add(BatchNormalization())
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1, activation='sigmoid'))
    
    optimizer = Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return model

# Create the KerasClassifier wrapper with the `dropout_rate` parameter included
model = KerasClassifier(build_fn=create_improved_model, neuron=16, layers=1, dropout_rate=0.2, verbose=1)

# Define the grid search parameters
param_grid = {
    'neuron': [32, 64, 128],
    'layers': [1, 2, 3, 4],
    'epochs': [100, 150, 200],
    'dropout_rate': [0.2, 0.3, 0.4]  # Now included correctly
}

# Perform the grid search
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, verbose=1)
grid_result = grid.fit(X_train, y_train)

# Print the best parameters
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


Fitting 3 folds for each of 108 candidates, totalling 324 fits


c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\joblib\externals\loky\process_executor.py:752: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  warnings.warn(


Epoch 1/200


c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\scikeras\wrappers.py:925: UserWarning: ``build_fn`` will be renamed to ``model`` in a future release, at which point use of ``build_fn`` will raise an Error instead.
  X, y = self._initialize(X, y)
c:\Users\naved\OneDrive\Desktop\Naved\Udemy\venv\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


11/11 ━━━━━━━━━━━━━━━━━━━━ 4s 9ms/step - accuracy: 0.4853 - loss: 0.9800 
Epoch 2/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.5458 - loss: 0.9485
Epoch 3/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4629 - loss: 0.9726
Epoch 4/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5261 - loss: 0.9731 
Epoch 5/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.5395 - loss: 0.8425 
Epoch 6/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.5470 - loss: 0.8325 
Epoch 7/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.4692 - loss: 0.8991 
Epoch 8/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.4969 - loss: 0.8656
Epoch 9/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.5465 - loss: 0.8518 
Epoch 10/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.5267 - loss: 0.8195
Epoch 11/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - accuracy: 0.6101 - loss: 0.7067
Epoch 12/200
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - accuracy: 0.53